In [34]:
import os
import math
import datetime
import numpy as np
import time
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import argparse
from torch import optim
from torch.autograd import Variable
import torchvision.utils

#from dataloader import data_loader
#from evaluation import evaluation_metrics

import torch.nn.functional as F
import numpy as np
import random
import pandas as pd

# from model import SiameseNetwork
#from model import Arcface, MobileFaceNet

import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils import data
import torchvision.transforms as transforms

In [36]:
class CustomDataset(data.Dataset):
    def __init__(self, root, phase='train',repeat_num = 1, transform=None):
        self.root = root
        self.phase = phase
        self.labels = {}
        self.transform = transform
        self.repeat_num = repeat_num
        if self.phase != 'train':
            self.label_path = os.path.join(root, self.phase, self.phase + '_label.csv')
            # used to prepare the labels and images path
            self.direc_df = pd.read_csv(self.label_path)
            self.direc_df.columns = ["image1", "image2", "label"]
            self.dir = os.path.join(root, self.phase)
        else:
            self.train_meta_dir = os.path.join(root, self.phase, self.phase + '_meta.csv')
            train_meta = pd.read_csv(self.train_meta_dir)

            train_data = []
            # make_true_pair
            id_list = list(set(train_meta['face_id']))
            for i in range(int(self.repeat_num)):
                for id in id_list:
                    pair = []
                    candidate = train_meta[train_meta['face_id'] == int(id)]
                    pair.append(candidate[candidate['cam_angle']=='front'].sample(1)['file_name'].item())
                    pair.append(candidate[candidate['cam_angle']=='side'].sample(1)['file_name'].item())
                    pair.append(0)
                    train_data.append(pair)
            # make_false_pair
            id_list = list(set(train_meta['face_id']))
            for i in range(int(self.repeat_num)):
                for id in id_list:
                    pair = []
                    candidate = train_meta[train_meta['face_id'] == int(id)]
                    candidate_others = train_meta[train_meta['face_id'] != int(id)]
                    pair.append(candidate[candidate['cam_angle']=='front'].sample(1)['file_name'].item())
                    pair.append(candidate_others[candidate_others['cam_angle']=='side'].sample(1)['file_name'].item())
                    pair.append(1)
                    train_data.append(pair)
            self.direc_df = pd.DataFrame(train_data)
            self.direc_df.columns = ["image1", "image2", "label"]
            self.dir = os.path.join(root, self.phase)
            self.direc_df.to_csv(os.path.join(root, self.phase, f'{self.phase}_label_{repeat_num}.csv'), mode='w', index=False)
            self.label_path = os.path.join(root, self.phase, f'{self.phase}_label_{repeat_num}')
            
    def __getitem__(self, index):
        # getting the image path
        image1_path = os.path.join(self.dir, self.direc_df.iat[index, 0])
        image2_path = os.path.join(self.dir, self.direc_df.iat[index, 1])
        # Loading the image
        img0 = Image.open(image1_path)
        img1 = Image.open(image2_path)
        # img0 = img0.convert("RGB")
        # img1 = img1.convert("RGB")
        
        ############################
        ######## get img_id
        ############################
        img0_filename = os.path.basename(image1_path)
        img0_id = os.path.splitext(img0_filename)[0].split('_')[0]
        img1_filename = os.path.basename(image1_path)
        img1_id = os.path.splitext(img1_filename)[0].split('_')[0]
        

        # Apply image transformations
        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        if self.phase != 'test':
            return (self.direc_df.iat[index, 0], img0, img0_id,
                    self.direc_df.iat[index, 1], img1, img1_id,
                    torch.from_numpy(np.array([int(self.direc_df.iat[index, 2])], dtype=np.float32)))
        elif self.phase == 'test':
            dummy = ""
            return (self.direc_df.iat[index, 0], img0, self.direc_df.iat[index, 1], img1, dummy)

    def __len__(self):
        return len(self.direc_df)

    def get_label_file(self):
        print('label:', self.label_path)
        return self.label_path

def data_loader(root, phase='train', batch_size=64,repeat_num=10):
    if phase == 'train':
        shuffle = True
    else:
        shuffle = False

    trfs = transforms.Compose([
        transforms.Resize((112,112)),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ])
    
    dataset = CustomDataset(root, phase, repeat_num, trfs)
    dataloader = data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader, dataset.get_label_file()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [37]:
# for reproducibility
np.random.seed(777)
random.seed(777)
torch.manual_seed(777)
torch.cuda.manual_seed_all(777)

DATASET_PATH = os.path.join('../data/03_face_verification_angle/')
print(os.path.isdir(DATASET_PATH))

batch = 64

True


In [38]:
 # get data loader
train_dataloader, _ = data_loader(root=DATASET_PATH, phase='train', batch_size=batch)

label: ../data/03_face_verification_angle/train/train_label_10


In [41]:
for iter_, data in enumerate(train_dataloader, 0):
    iter1_, img0, img0_id, iter2_, img1, img1_id, label = data
    print(img0_id, type(img0_id), len(img0_id))
    print(img0.shape)
    img0, img1, label = img0.to(device), img1.to(device), label.to(device)

    #optimizer.zero_grad()

    #output1 = model(img0)
    #thetas1 = head(output1, img0_id )

    #output2 = model(img1)
    #thetas2 = head(output2, img1_id)


('18071103', '19082641', '19073022', '18072703', '19090321', '19083041', '18081003', '18071702', '19091733', '18072603', '19062421', '19081922', '18090501', '18072603', '19071922', '18091702', '19082722', '19101432', '19071211', '19083022', '19081432', '19092332', '17081701', '19100133', '18101901', '19072632', '18082101', '19071741', '19091122', '18072001', '17092201', '19091621', '18082701', '18062103', '19071241', '19080742', '19092611', '19082222', '18071803', '18082102', '19090912', '19071111', '18081602', '18062102', '19070431', '19070322', '19091631', '19071041', '18070903', '18100801', '19092641', '18072301', '19092621', '18070603', '19082822', '19071912', '19092611', '19100842', '19073122', '18082003', '17081603', '19092541', '19062732', '19082922') <class 'tuple'> 64
torch.Size([64, 3, 112, 112])
('19072621', '19081211', '18072502', '17083102', '18110101', '19090333', '18092003', '19070121', '19062832', '19091142', '18071101', '19090311', '17083103', '19070121', '17092103', '

('17092602', '19072632', '19082642', '18070601', '19082022', '18070601', '18082401', '19101512', '18081003', '19080542', '19070242', '19070412', '18082002', '17080902', '17082102', '17091201', '19092332', '18070502', '18090402', '19100842', '19072211', '19071211', '19101132', '18082803', '18071902', '18081701', '17092503', '17082801', '19090621', '19100811', '19062431', '18082102', '18101101', '19091711', '18091001', '17092602', '17081703', '19091813', '17081101', '18082202', '17083103', '19070531', '18070603', '19092332', '18081301', '19080141', '19090322', '19082621', '19080132', '18070902', '19091031', '18101201', '19100422', '19090912', '18082703', '18081701', '19091031', '19072241', '19062431', '19092522', '18100501', '19082812', '17082903', '19100231') <class 'tuple'> 64
torch.Size([64, 3, 112, 112])
('18062801', '19070412', '19071931', '19092611', '19072332', '19090532', '18082201', '18080801', '19101112', '18071902', '17091202', '19072332', '19082032', '17090502', '19080641', '

('17092002', '19082721', '17082801', '17081803', '19080843', '18062901', '17082103', '18080201', '18072401', '19081333', '18082203', '19080242', '19071211', '19100811', '18101002', '18070404', '19073012', '19082741', '19080641', '19072512', '19081921', '19071711', '19080742', '19080241', '19072541', '19082821', '18082401', '19100832', '19082642', '19070322', '19092332', '19100813', '18090601', '18101201', '19081441', '18070202', '19101513', '17091101', '18083001', '19080843', '19071922', '19100743', '18101701', '18091003', '19071231', '19082642', '19080221', '18091001', '18081301', '19080832', '17090503', '19082642', '18072601', '17082202', '18070902', '18101002', '19081411', '19081332', '17091102', '19101513', '19090211', '18101701', '17080803', '19090332') <class 'tuple'> 64
torch.Size([64, 3, 112, 112])
('19091141', '18071101', '19092332', '19100241', '19081342', '19080821', '18080801', '19100431', '18091301', '18082101', '19092432', '19082031', '19081421', '19091931', '17091203', '

('18080801', '19070142', '17082103', '18100301', '18070501', '19071532', '18071703', '17091903', '17080803', '18072403', '17080903', '19080641', '17092602', '18092101', '17081101', '19070431', '19072541', '19070912', '17081602', '18100202', '19071732', '19082842', '18101002', '19071021', '18082803', '18100502', '19070331', '18070603', '19073022', '19082331', '19091813', '19083041', '19091813', '19062531', '19071742', '18070901', '19062722', '18062602', '19072221', '19072541', '17082802', '17081602', '19070312', '19080542', '19070421', '19072422', '18070901', '17083102', '18090501', '19081422', '18080203', '19071931', '18082202', '18071301', '17080802', '19090621', '19100231', '18091304', '18083002', '18062001', '19082342', '19101041', '19071833', '19090341') <class 'tuple'> 64
torch.Size([64, 3, 112, 112])
('18091103', '19071832', '17082202', '19072312', '18082301', '19073122', '18071201', '19082843', '17091801', '19071611', '19091813', '18080203', '18082003', '19091731', '17081602', '

('18071201', '18091102', '18091802', '18082801', '18110101', '18082702', '19082721', '17090502', '19091731', '19090331', '19092311', '18071301', '19101112', '19082031', '19081432', '19082212', '19090421', '19090331', '18072403', '19081332', '18070403', '17091904', '19090911', '17083102', '19082131', '18082003', '19070421', '19072641', '17092703', '17091204', '18070302', '19091921', '19100232', '19072432', '19072541', '19082712', '19090313', '19072911', '17081601', '19080642', '18072503', '18062802', '18062802', '19100811', '17082401', '17082201', '19071711', '19100422', '18062802', '17082402', '17083101', '19080821', '19091611', '19071621', '19082931', '19072542', '19070912', '19100241', '18072002', '17092502', '18083101', '17091801', '19090621', '19071912') <class 'tuple'> 64
torch.Size([64, 3, 112, 112])
('19062621', '19082031', '18072502', '19090222', '19091013', '19081333', '19082821', '18080801', '17091101', '19090531', '19091831', '19090621', '19081441', '19081333', '19082931', '

...

('19071542', '19082111', '19072322', '19072341', '19081332', '18083001', '19101111', '19071211', '19092622', '19101421', '19070242', '19100831', '19091621', '19082931', '17082502', '19080543', '19080921', '18081301', '19092042', '19090612', '19071833', '19070221', '19072922', '17082202', '18070901', '17080902', '18062901', '17082902', '18071101', '19081333', '19101112', '19070242', '19071041', '19082841', '19092621', '18061801', '19062732', '18070901', '17082401', '19071141', '19080221', '19092031', '18082801', '17082103', '19072911', '19101431', '19081421', '19091741', '19072422', '18070902', '17081702', '19100133', '18091901', '19071732', '19091942', '19091733', '19092641', '19062622', '17092503', '19072632', '19091733', '19081333', '18080101', '19062531') <class 'tuple'> 64
torch.Size([64, 3, 112, 112])
('19062641', '19100231', '19080843', '17092702', '19071922', '19100232', '19072911', '18090601', '18080803', '19062842', '17082102', '19062641', '17083001', '18090603', '17082201', '

('19090211', '18080103', '19091921', '18090401', '17091503', '17080903', '19092341', '18083001', '19100833', '19073012', '19092312', '18091304', '18080801', '18071803', '19080241', '18081701', '19081313', '17091801', '18072701', '18082202', '19072641', '18071101', '18090301', '17091103', '18090703', '19083041', '19070912', '17080801', '19090333', '18080102', '18070602', '19071542', '19083032', '19082842', '19091733', '18061802', '18081602', '18070902', '19082112', '18091102', '18082903', '18080803', '19091721', '19080142', '19101512', '19071231', '19092521', '19100833', '19062832', '18101002', '18071102', '19081222', '18062103', '18090502', '19080542', '18072702', '19062531', '19062431', '19101432', '19062722', '19062732', '19071532', '17091202', '19080822') <class 'tuple'> 64
torch.Size([64, 3, 112, 112])
('19091731', '19101421', '19082212', '19090311', '19081423', '19101512', '19070331', '18062603', '19092611', '18090401', '18090702', '19091741', '19090331', '19092332', '18072302', '

('19100842', '18082903', '19081622', '18070203', '18101901', '18062101', '19072641', '18082703', '18080202', '18110101', '19082222', '19070531', '19090541', '18082703', '19082112', '17082801', '19101111', '19091122', '18091702', '19071211', '18082602', '18081003', '18070501', '19072542', '18082903', '19100711', '19081312', '19100812', '18092001', '18100301', '18062603', '18081303', '19080542', '17091203', '19092611', '19082032', '17091103', '19070431', '19080932', '19090532', '18100202', '19070522', '18061901', '19071522', '18072001', '19070222', '19100412', '19082321', '19072332', '19082212', '18062603', '19082612', '18071201', '18070901', '17080903', '19072221', '18101101', '19062641', '18073104', '19072212', '19070321', '19062842', '19081211', '19090322') <class 'tuple'> 64
torch.Size([64, 3, 112, 112])
('18091902', '18091001', '19071931', '19100422', '19100442', '19071833', '18071101', '18070403', '19091742', '19070912', '18081701', '19101421', '19101012', '19082013', '19071621', '

('19082112', '18072503', '19101012', '19072541', '19092331', '17081601', '18091802', '19080531', '19100812', '18062002', '19082022', '19101432', '17090401', '18081702', '19062531', '19080641', '18101901', '19071931', '17083101', '17083102', '18072001', '18082301', '18101901', '19072322', '19070121', '18101701', '19082233', '19070242', '18090702', '19082321', '19082232', '18090502', '17081702', '19082233', '19092311', '18081702', '19092422', '18080101', '19071021', '17082902', '19082721', '19092541', '19071922', '19071742', '19070831', '18062802', '18080803', '18080803', '17091201', '18062902', '19082212', '17082401', '19101421', '17091503', '19091711', '17090101', '19090532', '18062901', '18081702', '19081442', '19092312', '19062832', '19071021', '19070212') <class 'tuple'> 64
torch.Size([64, 3, 112, 112])
('17090102', '18082101', '19100721', '17082802', '17081701', '19062842', '19071522', '18101101', '17091203', '19092032', '19072211', '19092522', '19091131', '19080543', '18091304', '

('19082931', '19081942', '19082931', '18062901', '18082703', '18072601', '18080801', '17092102', '19071021', '17082902', '19070221', '19090333', '18082202', '19091131', '19090541', '18082803', '17082103', '18092003', '19082642', '19090331', '18082202', '19072432', '18072701', '19072322', '19091031', '18070901', '19072612', '17080903', '19062841', '19081312', '19070541', '19071832', '18080201', '17081602', '18082002', '18080904', '18100301', '19091131', '18062101', '18062702', '18070603', '18071702', '17083102', '18070601', '19081921', '17090501', '18073002', '19091142', '19090313', '19082621', '19071621', '18100502', '18091802', '18071703', '19062622', '19090332', '19070221', '19071141', '18082401', '19101012', '18062103', '19092611', '18070501', '18071103') <class 'tuple'> 64
torch.Size([64, 3, 112, 112])
('19100422', '18062103', '19081432', '19101513', '19071711', '19091141', '19081331', '19090541', '19100431', '19091733', '18072703', '19071241', '18100501', '19081331', '19071041', '